# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [19]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [20]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]
#df.columns = ['Title', 'Year', 'Synopsis', 'Review', ]
#df['original_lang']='en'


def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    #frames = list()
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #TODO 1: Add your code here
    df_eng.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_eng['original_lang']='en'
    #frames.append(df_eng)

    df_fr = pd.read_csv('data/movie_reviews_fr.csv')
    df_fr.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_fr['original_lang']='fr'
    #frames.append(df_fr)
    
    df_sp = pd.read_csv('data/movie_reviews_sp.csv')
    df_sp.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_sp['original_lang']='sp'
    #frames.append(df_sp)

    frames = [df_eng, df_fr, df_sp]

    df = pd.concat(frames,ignore_index=True)
    
    return df

df = preprocess_data()

In [21]:
df.sample(10)

,Title,Year,Synopsis,Review,original_lang
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",sp
14,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",fr
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",sp
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",sp
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",en
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",fr


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [22]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer =  MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer =  MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [23]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df["original_lang"] == "fr","Review"]
fr_reviews_en = [translate(i, fr_en_model, fr_en_tokenizer) for i in fr_reviews] 

# filter synopsis in French and translate to English
fr_synopsis = df.loc[df["original_lang"] == "fr","Synopsis"]
fr_synopsis_en = [translate(i, fr_en_model, fr_en_tokenizer) for i in fr_synopsis] 


# filter reviews in Spanish and translate to English
es_reviews = df.loc[df["original_lang"] == "sp","Review"]
es_reviews_en = [translate(i, es_en_model, es_en_tokenizer) for i in es_reviews]

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df["original_lang"] == "sp","Synopsis"]
es_synopsis_en = [translate(i, es_en_model, es_en_tokenizer) for i in es_synopsis]

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df["original_lang"] == "fr","Review"]= fr_reviews_en
df.loc[df["original_lang"] == "fr","Synopsis"] = fr_synopsis_en
df.loc[df["original_lang"] == "sp","Review"]= es_reviews_en
df.loc[df["original_lang"] == "sp","Synopsis"] = es_synopsis_en


/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [24]:
df.sample(10)

,Title,Year,Synopsis,Review,original_lang
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",sp
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",fr
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",fr


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [25]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    
    """
    result = classifier(text)
    return result

In [26]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
reviews = df["Review"]
sentiment_result = [analyze_sentiment(i, sentiment_classifier) for i in reviews]
df['sentiment']=sentiment_result

In [27]:
df.sample(10)

,Title,Year,Synopsis,Review,original_lang,sentiment
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en,"[{'label': 'NEGATIVE', 'score': 0.999798834323..."
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",fr,"[{'label': 'POSITIVE', 'score': 0.999891877174..."
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr,"[{'label': 'POSITIVE', 'score': 0.999892711639..."
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en,"[{'label': 'NEGATIVE', 'score': 0.999816358089..."
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp,"[{'label': 'NEGATIVE', 'score': 0.999818861484..."
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",fr,"[{'label': 'NEGATIVE', 'score': 0.999798476696..."
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr,"[{'label': 'NEGATIVE', 'score': 0.999813973903..."
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr,"[{'label': 'POSITIVE', 'score': 0.999881267547..."
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en,"[{'label': 'POSITIVE', 'score': 0.999881744384..."
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp,"[{'label': 'POSITIVE', 'score': 0.999887228012..."


In [28]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)